In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import itertools

from tkinter import *
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import Image,ImageTk
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, classification_report
import math

In [2]:
root = Tk()
root.geometry("1360x768")
root.maxsize(1300, 630)
root['bg']= '#49A'

In [3]:
bg_app_raw = Image.open("D:/PICTURE/Death-Is-Not-the-End.jpg")

# Resize the image in the given (width, height)
img_app_raw=bg_app_raw.resize((1360, 800))

# Conver the image in TkImage
bg_app = ImageTk.PhotoImage(img_app_raw)
#bg_app = PhotoImage(file="E:/KULIAHKULIAH/Sem 8/TA/Background app/579139.png")

cnvs1 = Canvas(root, width=900, height=800)
cnvs1.pack(fill="both")
cnvs1.create_image(0,0, image=bg_app, anchor ="nw")

# Declaration of Additional Variable
cnvs1.create_text(650,35, text=" Machine Learning Based Myocardial Ishcemia Classifications  ", fill='white', font=('times', 20, 'bold'))

#Add Logo ITS
image1 = Image.open("D:/PICTURE/Logo-ITS-Institut-Teknologi-Sepuluh-November-Original.png")
image1 = image1.resize((150,150))
logo1 = ImageTk.PhotoImage(image1)
cnvs1.create_image(20, 20, anchor="nw", image=logo1)

#Add Logo BME
image2 = Image.open("D:/PICTURE/bme logo.png")
image2 = image2.resize((130,130))
logo2 = ImageTk.PhotoImage(image2)
cnvs1.create_image(1140, 25, anchor="nw", image=logo2) 

4

In [4]:
base_frame = Frame(root, width= 800, height=500)
base_frame.place(x=240, y=60)

notebook = ttk.Notebook(base_frame)
notebook.pack()

frame1 = ttk.Frame(notebook, width=800, height=500)
frame1.pack()
frame1.pack_propagate(0)
notebook.add(frame1, text='Input')

frame2 = ttk.Frame(notebook, width=800, height=500)
frame2.pack()
frame2.pack_propagate(0)
notebook.add(frame2, text='Training')

frame3 = ttk.Frame(notebook, width=800, height=500)
frame3.pack()
frame3.pack_propagate(0)
notebook.add(frame3, text='Testing')

In [5]:
def OpenFile():
    global data_frame
    data_frame = askopenfilename(initialdir="G:/Semester 8/TA/DWT/Win32/Debug", title="Select A File", filetypes=(("csv files", "*.csv"),("all files", "*.*")))
    data_frame = pd.read_csv(data_frame, header=None)
    data_frame = data_frame.rename({250: 'Class'}, axis=1)
    var0.set("({} x {})".format(data_frame.shape[0],data_frame.shape[1]))

In [6]:
def SplitData():
    global X_train, Y_train, X_val, Y_val
    train_data, val_data = train_test_split(data_frame, test_size=0.15)
    
    X_train = train_data.loc[:, train_data.columns != 'Class']
    Y_train = train_data.loc[:, train_data.columns == 'Class']
    Y_train = to_categorical(Y_train)
    var1.set("X_train ({} x {})".format(X_train.shape[0],X_train.shape[1]))    
    var2.set("Y_train ({} x {})".format(Y_train.shape[0],Y_train.shape[1]))

    X_val = val_data.loc[:, val_data.columns != 'Class']
    Y_val = val_data.loc[:, val_data.columns == 'Class']
    Y_val = to_categorical(Y_val)
    var3.set("X_Val ({} x {})".format(X_val.shape[0],X_val.shape[1]))
    var4.set("Y_Val ({} x {})".format(Y_val.shape[0],Y_val.shape[1]))

    train_size = train_data['Class'].value_counts()
    fig1 = Figure(figsize=(5,5)) # create a figure object
    ax1 = fig1.add_subplot() # add an Axes to the figure
    ax1.pie(train_size, labels=['Normal','Ischemia'], colors=['blue','skyblue'],autopct='%1.1f%%',radius=1.2,textprops={'fontsize': 10})     
    pie1 = FigureCanvasTkAgg(fig1, in_frame2)
    pie1.get_tk_widget().pack(side=LEFT, fill=BOTH, expand=1)

    val_size = val_data['Class'].value_counts()
    fig2 = Figure(figsize=(5,5))
    ax2 = fig2.add_subplot() # add an Axes to the figure
    ax2.pie(val_size, labels=['Normal','Ischemia'], colors=['red','orange'],autopct='%1.1f%%',radius=1.2,textprops={'fontsize': 10})     
    pie2 = FigureCanvasTkAgg(fig2, in_frame2)
    pie2.get_tk_widget().pack(side=LEFT, fill=BOTH, expand=1)

    Label7 = Label(in_frame3, text=" Data Size Training ", bg="white")
    Label7.pack(side=LEFT, ipadx=147)
    Label8 = Label(in_frame3, text=" Data Size Validation ", bg="white")
    Label8.pack(side=LEFT, ipadx=147)


In [7]:
def Training():
    global ann_model
    ann_model = Sequential()
    ann_model.add(Dense(32, activation='relu', input_shape=(250,)))
    ann_model.add(Dense(24, activation='relu'))
    ann_model.add(Dense(2, activation='softmax'))
    ann_model.summary()    
    
    ann_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    #my_callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    epocs = 300   
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_begin(self, epoch, logs={}):
            speed = 1
            pbar['value'] += (speed/epocs)*100                
            percentLabel.config(text=str(math.ceil(pbar['value'])) + "%")                                                  
            in_frame1_21.update_idletasks()            
    my_callbacks = myCallback()

    history = ann_model.fit(
        X_train,Y_train,
        validation_data=(X_val, Y_val),
        batch_size=32,
        epochs=epocs,
        callbacks=[my_callbacks]
    )   

    taskLabel.config(text="Completed !")

    my_acc = history.history['accuracy']
    my_val_acc = history.history['val_accuracy']

    # plot accuracy
    fig3 = Figure(figsize=(5, 5))
    ax3 = fig3.add_subplot()    
    ax3.set_title('Training Accuracy')    
    ax3.set_ylabel('Accuracy')
    ax3.set_xlabel('Epochs')
    ax3.set_ylim([min(ax3.set_ylim()),1])
    ax3.plot(my_acc, label='Training')
    ax3.plot(my_val_acc, label='Validation')  
    ax3.legend(loc='lower right') 
    line1 = FigureCanvasTkAgg(fig3, in_frame1_23)
    line1.get_tk_widget().pack(side=LEFT, fill=BOTH, expand=1)    

    # visualize the loss training history
    my_loss = history.history['loss']
    my_val_loss = history.history['val_loss']

    # plot loss
    fig4 = Figure(figsize=(5, 5))
    ax4 = fig4.add_subplot()
    ax4.set_title('Training Loss')
    ax4.set_ylabel('Loss')
    ax4.set_xlabel('Epochs')    
    ax4.set_ylim([min(ax4.set_ylim()),1])
    ax4.plot(my_loss, label='Training')
    ax4.plot(my_val_loss, label='Validation')
    ax4.legend(loc='lower left')
    line2 = FigureCanvasTkAgg(fig4, in_frame1_23)
    line2.get_tk_widget().pack(side=LEFT, fill=BOTH, expand=1)

In [8]:
def OpenDataTest():
     global test_data
     test_data = askopenfilename(initialdir="G:/Semester 8/TA/DWT/Win32/Debug", title="Select A File", filetypes=(("csv files", "*.csv"),("all files", "*.*")))
     test_data = pd.read_csv(test_data, header=None)
     test_data = test_data.rename({250: 'Class'}, axis=1)
     var5.set("({} x {})".format(test_data.shape[0],test_data.shape[1]))

In [9]:
def SplitDataTest():
    global X_test, Y_test
    X_test = test_data.loc[:, test_data.columns != 'Class']
    Y_test = test_data.loc[:, test_data.columns == 'Class']
    Y_test = to_categorical(Y_test)
    var6.set("X_test ({} x {})".format(X_test.shape[0],X_test.shape[1]))
    var7.set("Y_test ({} x {})".format(Y_test.shape[0],Y_test.shape[1]))

    test_size = test_data['Class'].value_counts()
    fig5 = Figure(figsize=(5,5))
    ax5 = fig5.add_subplot() # add an Axes to the figure
    ax5.pie(test_size, labels=['Normal','Ischemia'], colors=['green','lightgreen'],autopct='%1.1f%%',radius=1.2,textprops={'fontsize': 10})     
    pie3 = FigureCanvasTkAgg(fig5, in_frame2_3_1)
    pie3.get_tk_widget().pack(side=TOP, expand=1)

    Label8 = Label(in_frame2_3_1, text=" Data Size Testing ", bg="white")
    Label8.pack(side=TOP, ipadx=147)

In [10]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    print(' ')

    fig6 = Figure(figsize=(5,5))
    ax6 = fig6.add_subplot()
    ax6.imshow(cm, interpolation='nearest', cmap=cmap)
    ax6.set_title(title)
    #ax6.colorbar()
    tick_marks = np.arange(len(classes))
    #ax6.set_xticklabels(tick_marks, classes)
    #ax6.set_yticklabels(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax6.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    ax6.set_ylabel('True label')
    ax6.set_xlabel('Classified label')
    #ax6.tight_layout()
    confusion_chart = FigureCanvasTkAgg(fig6, in_frame2_3_2)
    confusion_chart.get_tk_widget().pack(side=TOP, padx=5, expand=1)

In [11]:
def TestingModel():
    hasil_evaluasi = ann_model.evaluate(X_test, Y_test, batch_size=32)
    var8.set("Accuracy : {:.2f}%".format(hasil_evaluasi[1] * 100))
    var9.set("Loss : {:.2f}%".format(hasil_evaluasi[0] * 100))

    Y_pred = ann_model.predict(X_test)
    Y_pred = np.argmax(Y_pred,axis=1)
    Y_true = np.argmax(Y_test,axis=1)
    cm = confusion_matrix(Y_true,Y_pred)
    plot_confusion_matrix(cm, classes=['Normal', 'Ischemia'])  
    print(type(classification_report(Y_true,Y_pred)))


In [12]:
#Frame1 "Input" Menu
in_frame1 = Frame(frame1, width=800, height=60, bg="white")
in_frame1.pack()
in_frame1.pack_propagate(0)

var0 = StringVar()
var1 = StringVar()
var2 = StringVar()
var3 = StringVar()
var4 = StringVar()

btn1 = Button(in_frame1, text="Load File", command=OpenFile)
btn1.pack(side=LEFT, ipadx=5, padx=20)
btn2 = Button(in_frame1, text="Split Data", command=SplitData)
btn2.pack(side=LEFT, ipadx=5, padx=20)

box0  = LabelFrame(in_frame1, text="Dataset Frame", bg="white")
box0.pack(side=LEFT, padx=5)
Label0 = Label(box0, text="", textvariable=var0, bg="white")
Label0.pack(side=TOP)

box1  = LabelFrame(in_frame1, text="Training Data Frame", bg="white")
box1.pack(side=LEFT, padx=5)
Label1 = Label(box1, text="", textvariable=var1, bg="white")
Label1.pack(side=TOP)
Label2 = Label(box1, text="", textvariable=var2, bg="white")
Label2.pack(side=TOP)

box2  = LabelFrame(in_frame1, text="Validation Data Frame", bg="white")
box2.pack(side=LEFT, padx=5)
Label3 = Label(box2, text="", textvariable=var3, bg="white")
Label3.pack(side=TOP)
Label4 = Label(box2, text="", textvariable=var4, bg="white")
Label4.pack(side=TOP)

in_frame2 = Frame(frame1, width=800, height=400, bg="white")
in_frame2.pack(side=TOP)
in_frame2.pack_propagate(0)

in_frame3 = Frame(frame1, width=800, height=40, bg="white")
in_frame3.pack(side=TOP)
in_frame3.pack_propagate(0)


In [13]:
#FRAME 2 "Training" Menu

in_frame1_2 = Frame(frame2, width=800, height=40, bg="white")
in_frame1_2.pack()
in_frame1_2.pack_propagate(0)

btn3 = Button(in_frame1_2, text="Training Model", command=Training)
btn3.pack(side=LEFT, padx=15)

in_frame1_21 = Frame(in_frame1_2, width=700, height=40, bg="white")
in_frame1_21.pack(side=RIGHT)
in_frame1_21.pack_propagate(0)

pbar = ttk.Progressbar(in_frame1_21,orient = HORIZONTAL,length = 700, mode = 'determinate')
pbar.pack(side=LEFT)

in_frame1_22 = Frame(frame2, width=800, height=20, bg="white")
in_frame1_22.pack()
in_frame1_22.pack_propagate(0)

in_frame1_22_1 = Frame(in_frame1_22, width=400, height=20, bg="white")
in_frame1_22_1.pack(side=LEFT)
in_frame1_22_1.pack_propagate(0)

in_frame1_22_2 = Frame(in_frame1_22, width=400, height=20, bg="white")
in_frame1_22_2.pack(side=LEFT)
in_frame1_22_2.pack_propagate(0)

percentLabel = Label(in_frame1_22_1, bg="white")
percentLabel.pack(side=RIGHT)

taskLabel = Label(in_frame1_22_2, bg="white")
taskLabel.pack(side=LEFT)

in_frame1_23 = Frame(frame2, width=800, height=440, bg="white")
in_frame1_23.pack()
in_frame1_23.pack_propagate(0)

In [14]:
#Frame 3 "Testing" Menu

in_frame1_3 = Frame(frame3, width=800, height=60, bg="white")
in_frame1_3.pack()
in_frame1_3.pack_propagate(0)

btn4 = Button(in_frame1_3, text="Load Data Test", command=OpenDataTest)
btn4.pack(side=LEFT, ipadx=5, padx=20)
btn5 = Button(in_frame1_3, text="Split Data", command=SplitDataTest)
btn5.pack(side=LEFT, ipadx=5, padx=20)
btn6 = Button(in_frame1_3, text="Testing", command=TestingModel)
btn6.pack(side=LEFT, ipadx=5, padx=20)

var5 = StringVar()
var6 = StringVar()
var7 = StringVar()
var8 = StringVar()
var9 = StringVar()

box3  = LabelFrame(in_frame1_3, text="Data Test Frame", bg="white")
box3.pack(side=LEFT, padx=10)
Label5 = Label(box3, text="", textvariable=var5, bg="white")
Label5.pack(side=TOP)

box4  = LabelFrame(in_frame1_3, text="Testing Data Frame", bg="white")
box4.pack(side=LEFT, padx=10)
Label6 = Label(box4, textvariable=var6, bg="white")
Label6.pack(side=TOP)
Label7 = Label(box4, textvariable=var7, bg="white")
Label7.pack(side=TOP)

box5  = LabelFrame(in_frame1_3, text="Testing Result", bg="white")
box5.pack(side=LEFT, padx=10)
Label9 = Label(box5, textvariable=var8, bg="white")
Label9.pack(side=TOP)
Label10 = Label(box5, textvariable=var9, bg="white")
Label10.pack(side=TOP)

in_frame2_3 = Frame(frame3, width=800, height=440)
in_frame2_3.pack()
in_frame2_3.pack_propagate(0)

in_frame2_3_1 = Frame(in_frame2_3, width=400, height=440, bg="white")
in_frame2_3_1.pack(side=LEFT)
in_frame2_3_1.pack_propagate(0)

in_frame2_3_2 = Frame(in_frame2_3, width=400, height=440, bg="white")
in_frame2_3_2.pack(side=LEFT)
in_frame2_3_2.pack_propagate(0)

In [15]:
root.mainloop()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                8032      
                                                                 
 dense_1 (Dense)             (None, 24)                792       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 8,874
Trainable params: 8,874
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
83/83 [==============================] - 3s 10ms/step - loss: 0.7390 - accuracy: 0.4885 - val_loss: 0.7289 - val_accuracy: 0.4989
Epoch 2/300
83/83 [==============================] - 1s 7ms/step - loss: 0.7289 - accuracy: 0.4964 - val_loss: 0.7193 - val_accuracy: 0.5032
Epoch 3/300
83/83 [================